## 146. LRU Cache
- Description:
  <blockquote>
    Design a data structure that follows the constraints of a **[Least Recently Used (LRU) cache](https://en.wikipedia.org/wiki/Cache_replacement_policies#LRU)**.

  Implement the `LRUCache` class:

  -   `LRUCache(int capacity)` Initialize the LRU cache with **positive** size `capacity`.
  -   `int get(int key)` Return the value of the `key` if the key exists, otherwise return `-1`.
  -   `void put(int key, int value)` Update the value of the `key` if the `key` exists. Otherwise, add the `key-value` pair to the cache. If the number of keys exceeds the `capacity` from this operation, **evict** the least recently used key.

  The functions `get` and `put` must each run in `O(1)` average time complexity.

  **Example 1:**

  ```
  Input
  ["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
  [[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]
  Output
  [null, null, null, 1, null, -1, null, -1, 3, 4]

  Explanation
  LRUCache lRUCache = new LRUCache(2);
  lRUCache.put(1, 1); // cache is {1=1}
  lRUCache.put(2, 2); // cache is {1=1, 2=2}
  lRUCache.get(1);    // return 1
  lRUCache.put(3, 3); // LRU key was 2, evicts key 2, cache is {1=1, 3=3}
  lRUCache.get(2);    // returns -1 (not found)
  lRUCache.put(4, 4); // LRU key was 1, evicts key 1, cache is {4=4, 3=3}
  lRUCache.get(1);    // return -1 (not found)
  lRUCache.get(3);    // return 3
  lRUCache.get(4);    // return 4

  ```

  **Constraints:**

  -   `1 <= capacity <= 3000`
  -   `0 <= key <= 10<sup>4</sup>`
  -   `0 <= value <= 10<sup>5</sup>`
  -   At most `2 * 10<sup>5</sup>` calls will be made to `get` and `put`.
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/lru-cache/description/)

- Topics: Linked List, OrderedDict

- Difficulty: Medium

- Resources: example_resource_URL

### Solution 1, Hash Map & Doubly Linked List


Hash Map where Key is key and value is a node of a Doubly Linked &
Doubly Linked List to maintain LRU order of Nodes with LRU node at tail and most recently used node at head

Why Doubly Linked List?  
In an LRU cache, when you access a node (via get or put), you often need to remove it from the middle and move it to the front.

- In a singly linked list, to remove a node, you need access to its previous node (to update prev.next). But if you only have a pointer to the current node, you can’t find the previous node in O(1) — you’d have to traverse from the head.

- In a doubly linked list, each node has a prev pointer, so you can remove any node in O(1) given just that node.


---

- Time Complexity: O(1) for both get and put.
  - For get:

    Check if a key is in a hash map. This costs O(1).  
    Get a node associated with a key. This costs O(1).  
    Call remove and add. Both methods cost O(1).

  - For put:

      Check if a key is in a hash map. This costs O(1).  
      If it is, we get a node associated with a key and call remove. Both cost O(1).  
      Create a new node and insert it into the hash map. This costs O(1).  
      Call add. This method costs O(1).  
      If the capacity is exceeded, we call remove and delete from the hash map. Both cost O(1).


- Space Complexity: O(capacity)
  - We use extra space for the hash map and for our linked list. Both cannot exceed a size of capacity.

In [ ]:
class ListNode:
    def __init__(self, key, val):
        self.key = key
        self.val = val
        self.next = None
        self.prev = None


class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.keyToNodeDict = {}
        self.head = ListNode(-1, -1)
        self.tail = ListNode(-1, -1)
        self.head.next = self.tail
        self.tail.prev = self.head

    def get(self, key: int) -> int:
        if key not in self.keyToNodeDict:
            return -1

        node = self.keyToNodeDict[key]
        self.remove(node)
        self.add(node)
        return node.val

    def put(self, key: int, value: int) -> None:
        if key in self.keyToNodeDict:
            old_node = self.keyToNodeDict[key]
            self.remove(old_node)

        node = ListNode(key, value)
        self.keyToNodeDict[key] = node
        self.add(node)

        if len(self.keyToNodeDict) > self.capacity:
            node_to_delete = self.head.next
            self.remove(node_to_delete)
            del self.keyToNodeDict[node_to_delete.key]

    # Most Recently Used node is added at the tail end
    # And Least Recently Used node is the one at the head of the doubly linked list
    def add(self, node):
        previous_end = self.tail.prev
        previous_end.next = node
        node.prev = previous_end
        node.next = self.tail
        self.tail.prev = node

    def remove(self, node):
        node.prev.next = node.next
        node.next.prev = node.prev


# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

### Solution 2, In built OrderedDict
Using a built in data structure that keeps track of the insertion order of the keys of a dict and allows you to move the key-value pair to either the front or the back to maintain LRU order.

It is similar to previous approach, The only difference is that much of the code we needed to implement the linked list logic is now done automatically for us by the built-in data structure we are using.



- Time complexity: O(1) for both get and put.
- Space complexity: O(capacity)


In [ ]:
import collections


class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.keyValueDict = collections.OrderedDict()

    def get(self, key: int) -> int:
        if key not in self.keyValueDict:
            return -1

        # The most recently used key, value pair is moved to the end
        # therefore the least recently used key,value pair will be at the beginning
        self.keyValueDict.move_to_end(key)
        return self.keyValueDict[key]

    def put(self, key: int, value: int) -> None:
        # If the key already exists in the dict, movie it to the end as it is the most recently used key
        if key in self.keyValueDict:
            self.keyValueDict.move_to_end(key)

        # then update it's value 
        # OR create a new key value pair that will automatically be at the end if the key did not already exist
        self.keyValueDict[key] = value
        
        if len(self.keyValueDict) > self.capacity:
            # removes and returns the first (least recently added/accessed) item → FIFO
            self.keyValueDict.popitem(False)


# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

In [ ]:
from collections import OrderedDict

# Create an OrderedDict
d = OrderedDict.fromkeys('abcde')
print("Original OrderedDict:", d)

# Move 'b' to the end
d.move_to_end('b')
print("After moving 'b' to the end:", d)

# Move 'b' to the beginning
d.move_to_end('b', last=False)
print("After moving 'b' to the beginning:", d)